In [1]:
import sys 
sys.path.append("..") 
from utils import config
data_type = 'Mixbig_Elect_30'
config.Data_path = '../Test-Data/%s/'%(data_type)
from utils.seq2seq.batcher import *
from utils.seq2seq.train_util import *
from translate.seq2seq_beam import *
from utils.seq2seq.write_result import *
import argparse
import pandas as pd

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 

parser = argparse.ArgumentParser()
parser.add_argument('--key_attention', type=bool, default=False, help = 'True/False')
parser.add_argument('--intra_encoder', type=bool, default=False, help = 'True/False')
parser.add_argument('--intra_decoder', type=bool, default=False, help = 'True/False')
parser.add_argument('--copy', type=bool, default=True, help = 'True/False') # for transformer

parser.add_argument('--model_type', type=str, default='seq2seq', choices=['seq2seq', 'transformer'])
parser.add_argument('--train_rl', type=bool, default=False, help = 'True/False')
parser.add_argument('--keywords', type=str, default='POS_keys', 
                    help = 'POS_keys / DEP_keys / Noun_adj_keys / TextRank_keys')

parser.add_argument('--lr', type=float, default=0.0001)
parser.add_argument('--rand_unif_init_mag', type=float, default=0.02)
parser.add_argument('--trunc_norm_init_std', type=float, default=0.001)
parser.add_argument('--mle_weight', type=float, default=1.0)
parser.add_argument('--gound_truth_prob', type=float, default=0.1)

parser.add_argument('--max_enc_steps', type=int, default=500)
parser.add_argument('--max_dec_steps', type=int, default=20)
parser.add_argument('--min_dec_steps', type=int, default=5)
parser.add_argument('--max_epochs', type=int, default=15)
parser.add_argument('--vocab_size', type=int, default=50000)
parser.add_argument('--beam_size', type=int, default=16)
parser.add_argument('--batch_size', type=int, default=1)

parser.add_argument('--hidden_dim', type=int, default=512)
parser.add_argument('--emb_dim', type=int, default=300)
parser.add_argument('--gradient_accum', type=int, default=1)

parser.add_argument('--word_emb_type', type=str, default='word2Vec', help='word2Vec/glove/FastText')
parser.add_argument('--pre_train_emb', type=bool, default=True, help = 'True/False') # 若pre_train_emb為false, 則emb type為NoPretrain

opt = parser.parse_args(args=[])
config = re_config(opt)


In [2]:
import os
if not os.path.exists(data_type):
    os.makedirs(data_type)


config.word_emb_path = config.word_emb_path.replace('Test-Data/Mixbig_Elect_30','Train-Data/Mix6_mainCat_Ekphrasis')
config.vocab_path = config.vocab_path.replace('Test-Data/Mixbig_Elect_30','Train-Data/Mix6_mainCat_Ekphrasis')
os.path.abspath(config.vocab_path)

'/home/eagleuser/Users/leyan/Summarize_parallel/Train-Data/Mix6_mainCat_Ekphrasis/Embedding/word2Vec/word.vocab'

In [3]:
loggerName, writerPath = getName(config)   
loggerName = loggerName + "_TEST"
# writerPath = writerPath.replace('Pointer-Generator','Pointer_less_Intra_dec')

logger = getLogger(loggerName)

2020-07-03 19:04:31 - Pointer_generator_word2Vec_TEST - INFO: - logger已啟動


In [4]:
_, validate_loader, vocab = getDataLoader(logger, config)
# train_batches = len(iter(train_loader))
test_batches = len(iter(validate_loader))
# save_steps = int(train_batches/250)*250
vocab

2020-07-03 19:08:10 - Pointer_generator_word2Vec_TEST - INFO: - train : 491432, test : 54604
2020-07-03 19:08:10 - Pointer_generator_word2Vec_TEST - INFO: - train batches : 491432, test batches : 54604


# Load Model

In [5]:
load_model_path = 'PG-less.tar'  


import torch as T

T.backends.cudnn.benchmark = True 
checkpoint = T.load(load_model_path)
# vocab = checkpoint['vocab']
print(load_model_path)

from create_model.pointer_less import Model 

model = Model(pre_train_emb=config.pre_train_emb, 
              word_emb_type = config.word_emb_type, 
              vocab = vocab)

model = model.cuda()
model.load_state_dict(checkpoint['model'])
model.eval()

PG-less.tar


I0703 19:08:14.203657 139797938509632 file_utils.py:35] PyTorch version 1.4.0 available.
/home/eagleuser/.conda/envs/Leyan/lib/python3.6/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
I0703 19:08:15.370535 139797938509632 utils_any2vec.py:341] loading projection weights from /home/eagleuser/Users/leyan//Train-Data/Mix6_mainCat_Ekphrasis/Embedding/word2Vec/word2Vec.300d.txt


/home/eagleuser/Users/leyan//Train-Data/Mix6_mainCat_Ekphrasis/Embedding/word2Vec/word2Vec.300d.txt


I0703 19:08:26.209514 139797938509632 utils_any2vec.py:405] loaded (48560, 300) matrix from /home/eagleuser/Users/leyan//Train-Data/Mix6_mainCat_Ekphrasis/Embedding/word2Vec/word2Vec.300d.txt


Model(
  (encoder): Encoder(
    (lstm): LSTM(300, 512, batch_first=True, dropout=0.2, bidirectional=True)
    (reduce_h): Linear(in_features=1024, out_features=512, bias=True)
    (reduce_c): Linear(in_features=1024, out_features=512, bias=True)
  )
  (decoder): Decoder(
    (enc_attention): MultiHeadedAttention(
      (linears): ModuleList(
        (0): Linear(in_features=1024, out_features=1024, bias=True)
        (1): Linear(in_features=1024, out_features=1024, bias=True)
        (2): Linear(in_features=1024, out_features=1024, bias=True)
        (3): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (dec_attention): decoder_attention()
    (x_context): Linear(in_features=1324, out_features=300, bias=True)
    (x_key_context): Linear(in_features=1624, out_features=300, bias=True)
    (lstm): LSTMCell(300, 512)
    (p_gen_linear): Linear(in_features=2860, out_features=1, bias=True)
    (p_gen_dropout): Dropout(p=0

In [6]:
writerPath = './'
import time

In [7]:
@torch.autograd.no_grad()
def decode(dataloader, epoch = 0):
    # 動態取batch
    num = len(dataloader)
    outFrame = None
    avg_time = 0
    total_scores = dict()   
    idx = 0 
    for _, inputs in enumerate(dataloader):
        start = time.time() 
        # 'Encoder data'
        enc_batch, enc_padding_mask, enc_lens, enc_batch_extend_vocab, extra_zeros, coverage,             ct_e, enc_key_batch, enc_key_mask, enc_key_lens = get_input_from_batch(inputs, config, batch_first = True)
        max_enc_len = max(T.max(enc_lens,dim=0)).tolist()[0] 
        
        if (max_enc_len != max(enc_lens.tolist())[0]): continue
        if len(enc_key_batch[0]) == 0: enc_key_batch = T.LongTensor([0]).cuda()
#         print(enc_key_batch[0],len(enc_key_batch[0]))
        enc_batch = model.embeds(enc_batch)  # Get embeddings for encoder input    
        enc_key_batch = model.embeds(enc_key_batch)  # Get key embeddings for encoder input

        enc_out, enc_hidden = model.encoder(enc_batch, enc_lens, max_enc_len)
        
        # 'Feed encoder data to predict'
        pred_ids = beam_search(enc_hidden, enc_out, enc_padding_mask, ct_e, extra_zeros, 
                                enc_batch_extend_vocab, enc_key_batch, enc_key_mask, model, 
                                START, END, UNKNOWN_TOKEN)

        article_sents, decoded_sents, keywords_list, ref_sents, long_seq_index = prepare_result(vocab, inputs, pred_ids)
        cost = (time.time() - start)
        avg_time += cost        

        multi_scores, batch_frame = total_evaulate(article_sents, keywords_list, decoded_sents, ref_sents)
        review_IDS = [review_ID for review_ID in inputs.review_IDS]
        batch_frame['review_ID'] = review_IDS        
        if idx %1000 ==0 and idx >0 : 
            print(idx); 
        if idx == 0: 
            outFrame = batch_frame; 
            total_scores = multi_scores
        else: 
            outFrame = pd.concat([outFrame, batch_frame], axis=0, ignore_index=True) 
            for key, scores in total_scores.items():
                scores.extend(multi_scores[key])
                total_scores[key] = scores
        idx += 1
        # ----------------------------------------------------    
    avg_time = avg_time / (num * config.batch_size)    

    scalar_acc = {}
    num = 0
    for key, scores in total_scores.items():
        num = len(scores)
        scalar_acc[key] = sum(scores)/len(scores)    
    
    # -----------------------------------------------------------
    total_output(epoch, 'test', writerPath, outFrame, avg_time, num , scalar_acc
    )
    # -----------------------------------------------------------
    outFrame = outFrame.sort_values(by=['rouge_l'], ascending=False)
    big_frame = outFrame.head()
    small_frame = outFrame.tail()    
    # -----------------------------------------------------------
    
    return outFrame, scalar_acc


In [8]:
test_outFrame, scalar_acc = decode(validate_loader)                   

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
test 
 ['Accuracy result:\n', '##-- Rouge-1 --##\n', 'testing_rouge_1_p: 0.1799647273455593 \n', 'testing_rouge_1_r: 0.17887940021151855 \n', 'testing_rouge_1_f: 0.1717402512585566 \n', '##-- Rouge-2 --##\n', 'testing_rouge_2_p: 0.04837252521019476 \n', 'testing_rouge_2_r: 0.049107616007987825 \n', 'testing_rouge_2_f: 0.046838219143360026 \n', '##-- Rouge-l --##\n', 'testing_rouge_l_p: 0.16109900861546503 \n', 'testing_rouge_l_r: 0.15915568388083812 \n', 'testing_rouge_l_f: 0.1437934898521938 \n', '##-- BLEU --##\n', 'testing_bleu_1: 0.13791245809169153 \n', 'testing_bleu_2: 0.047800219108112635 \n', 'testing_bleu_3: 0.03325125095974862 \n', 'testing_bleu_4: 0.0271891357218293

In [9]:
scalar_acc

{'rouge_1_p': 0.1799647273455593,
 'rouge_1_r': 0.17887940021151855,
 'rouge_1_f': 0.1717402512585566,
 'rouge_2_p': 0.04837252521019476,
 'rouge_2_r': 0.049107616007987825,
 'rouge_2_f': 0.046838219143360026,
 'rouge_l_p': 0.16109900861546503,
 'rouge_l_r': 0.15915568388083812,
 'rouge_l_f': 0.1437934898521938,
 'bleu_1': 0.13791245809169153,
 'bleu_2': 0.047800219108112635,
 'bleu_3': 0.03325125095974862,
 'bleu_4': 0.027189135721829308,
 'meteor': 0.1258603759712392}

In [10]:
path = '%s/output.xlsx'%(data_type)
writer = pd.ExcelWriter(path, engine='xlsxwriter')
#THIS
writer.book.use_zip64()
test_outFrame.to_excel(writer, index = False)
# Close the Pandas Excel writer and output the Excel file.
writer.save()
writer.close()
print(path + " Write finished")

Mixbig_Elect_30/output.xlsx Write finished


In [11]:
path = '%s/res.txt'%(data_type)
with open(path,'w',encoding='utf-8') as f:
    for k ,v in scalar_acc.items():
        f.write(k + ' : ' + str(v) + '\n')